In [ ]:
from utils import *

## configuration

In [ ]:
model = 'gpt2'
dataset = 'cola/raw/in_domain_dev.tsv'
number_of_shots = 4
prompt_set = '1'
device = 'cuda'

## load the model and prepare the data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model, torch_dtype=torch.float16).to(device)

df = pd.read_csv(dataset, sep='\t', names=['source', 'label', 'note', 'sentence'])
                
inputs_targets_g = prepare_data(df, 'cola', 'g', number_of_shots, prompt_set)
inputs_targets_g_prime = prepare_data(df, 'cola', 'g_prime', number_of_shots, prompt_set)

## calculate in-context PVI

### local

In [ ]:
torch.cuda.manual_seed_all(2266)
raw_responses, results = defaultdict(dict), defaultdict(list)
for i in trange(len(df)):
    results['input'].append(inputs_targets_g_prime[i][0])
    results['target'].append(inputs_targets_g_prime[i][1])
    raw_responses['g'][i], scores_g = predict(
        inputs_targets_g[i][0],
        model,
        tokenizer,
        max_lengths[model],
        max_new_tokens=2,
        pad_token_id=pad_ids[model],
        eos_token_id=None,
        temperature=0.8,
        device=device
    )
    raw_responses['g_prime'][i], scores_g_prime = predict(
        inputs_targets_g_prime[i][0],
        model,
        tokenizer,
        max_lengths[model],
        max_new_tokens=2,
        pad_token_id=pad_ids[model],
        eos_token_id=None,
        temperature=0.8,
        device=device
    )
    results['scores_g'].append([_.cpu().numpy() for _ in scores_g])
    results['scores_g_prime'].append([_.cpu().numpy() for _ in scores_g_prime])
    results['prediction_g'].append(raw_responses['g'][i].replace(inputs_targets_g[i][0], ''))
    results['prediction_g_prime'].append(raw_responses['g_prime'][i].replace(inputs_targets_g_prime[i][0], ''))

    target_id = tokenizer(' ' + results['target'][i]).input_ids[0]
    log_softmax_g = scores_g[0].squeeze().log_softmax(dim=0)
    log_softmax_g_prime = scores_g_prime[0].squeeze().log_softmax(dim=0)
    log_prob_g = float(log_softmax_g[target_id])
    log_prob_g_prime = float(log_softmax_g_prime[target_id])
    
    # replace -inf with the lowest log probability in log_softmax
    # if log_prob_g < -100:
    #     x = scores_g[0].squeeze().softmax(dim=0)
    #     target_id = torch.where(x > 0, x, x.max()).argmin()
    #     log_prob_g = float(log_softmax_g[target_id])
    # if log_prob_g_prime < -100:
    #     x = scores_g_prime[0].squeeze().softmax(dim=0)
    #     target_id = torch.where(x > 0, x, x.max()).argmin()
    #     log_prob_g_prime = float(log_softmax_g_prime[target_id])
    
    results['information_g'].append(log_prob_g)
    results['information_g_prime'].append(log_prob_g_prime)
    results['pvi'].append(-results['information_g'][i] + results['information_g_prime'][i])

### openai api

In [ ]:
openai.organization = ''
openai.api_key = ''

In [ ]:
raw_responses, results = defaultdict(dict), defaultdict(list)
for i in trange(len(df)):
    results['input'].append(inputs_targets_g_prime[i][0])
    results['target'].append(inputs_targets_g_prime[i][1])
    response_g = openai.Completion.create(
        prompt=inputs_targets_g[i][0],
        model=model,
        max_tokens=2,
        temperature=0.8,
        logprobs=5
    )
    response_g_prime = openai.Completion.create(
        prompt=inputs_targets_g_prime[i][0],
        model=model,
        max_tokens=2,
        temperature=0.8,
        logprobs=5
    )
    raw_responses['g'][i] = response_g
    raw_responses['g_prime'][i] = response_g_prime
    raw_responses['scores_g'][i] = raw_responses['g'][i]['choices'][0]['logprobs']['top_logprobs'][0]
    raw_responses['scores_g_prime'][i] = raw_responses['g_prime'][i]['choices'][0]['logprobs']['top_logprobs'][0]
    results['prediction_g'].append(raw_responses['g'][i]['choices'][0]['text'])
    results['prediction_g_prime'].append(raw_responses['g_prime'][i]['choices'][0]['text'])

    target = ' ' + results['target'][i]
    if target in raw_responses['scores_g'][i] and target in raw_responses['scores_g_prime'][i]:
        results['information_g'].append(raw_responses['scores_g'][i][target])
        results['information_g_prime'].append(raw_responses['scores_g_prime'][i][target])
        results['pvi'].append(-results['information_g'][i] + results['information_g_prime'][i])
    else:
        results['information_g'].append('na')
        results['information_g_prime'].append('na')
        results['pvi'].append('na')

## save the outputs

In [ ]:
with open('raw_responses.json', 'w') as file:
    json.dump(raw_responses, file, indent=4, ensure_ascii=False)

pd.DataFrame(results).to_csv('results.csv')